## Lab 10 - Clasificación de Perros y Gatos

Stefano Aragoni, Carol Arévalo, Luis Diego Santos

### Importar Librerías


In [1]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from imblearn.over_sampling import RandomOverSampler
import cv2
import random

## 1.1 Lectura del Dataset

### Lectura y Procesamiento de Imagenes

In [14]:

def load_images(path, size=(224, 224)):
    images = []
    labels = []

    dirs = ["Cat", "Dog"]

    for dir in dirs:

        contador = 0

        for file in os.listdir(path+'/'+dir):
            if file.endswith('.jpg'):
                img_path = os.path.join(path+'/'+dir, file)
                label = 1 if 'Dog' == dir else 0

                # Leer imagen a escala de grises
                img = cv2.imread(img_path)
                img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

                # Redimensionar imagen a tamaño específico
                img = cv2.resize(img, (500, 500))

                # Aplicar filtro bilateral para eliminar ruido
                img = cv2.bilateralFilter(img, 50, 30, 30)

                # Aplicar umbral para binarizar imagen
                ret, thresh = cv2.threshold(img, 150, 255, cv2.THRESH_BINARY)

                # Encontrar contornos
                contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

                # Dibujar contornos en imagen
                img = cv2.drawContours(img, contours, -1, (0, 255, 0), 1)

                if random.random() < 0.5:
                    
                    opt = random.randint(0, 3)


                    if opt == 0:
                        img = cv2.flip(img, 1)

                    elif opt == 1:
                        img = cv2.rotate(img, cv2.ROTATE_180)

                    elif opt == 2:
                        img = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)

                    elif opt == 3:
                        img = cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE)


                # Normalizar valores de píxeles
                img = img / 255.0

                # # show image on plot
                # plt.imshow(img, cmap='gray')
                # plt.show()

                images.append(img)
                labels.append(label)
                         
                if contador == 1000:
                    break

                if contador % 100 == 0:
                    print("Imagenes procesadas: ", contador)

                contador += 1

    return np.array(images), np.array(labels)

### Importar Imagenes y Split de Dataset

> Este proceso de lectura, a pesar de que solo se leen 1000 fotos de perros y 1000 de gatos, se tarda aprox 15 minutos.

In [15]:
# Definir ruta del conjunto de datos
data_path = './PetImages'

# Cargar imágenes y etiquetas
images, labels = load_images(data_path)

# Comprobar tamaños de los conjuntos
print('\nNúmero de imágenes por clase:', np.bincount(labels))

Imagenes procesadas:  0
Imagenes procesadas:  100
Imagenes procesadas:  200
Imagenes procesadas:  300
Imagenes procesadas:  400
Imagenes procesadas:  500
Imagenes procesadas:  600
Imagenes procesadas:  700
Imagenes procesadas:  800
Imagenes procesadas:  900
Imagenes procesadas:  0
Imagenes procesadas:  100
Imagenes procesadas:  200
Imagenes procesadas:  300
Imagenes procesadas:  400


In [ ]:
# Dividir conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Dividir 20 de prueba en 10 de validación y 10 de prueba
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

# Verificar tamaños de los conjuntos
print('\nTamaño del conjunto de entrenamiento:', X_train.shape)
print('Tamaño del conjunto de validación:', X_val.shape)
print('Tamaño del conjunto de prueba:', X_test.shape)


Tamaño del conjunto de entrenamiento: (1600,)
Tamaño del conjunto de validación: (200,)
Tamaño del conjunto de prueba: (200,)


## 1.2 Construccion del Modelo

In [ ]:
import tensorflow as tf

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])


In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_val, y_val))
